In [ ]:
%load_ext autoreload
import os, sys
sys.path.append(os.path.abspath('..'))

In [ ]:
import adaptive
import dependencies.adaptive_tools as adaptive_tools
adaptive.notebook_extension()
import holoviews as hv
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import cmath
import scipy.constants
from multiprocessing import Pool
from functools import partial
import kwant

import symmetry
import topology
import sns_system
import distributed_sns
import hpc05
import plotting_results
from copy import copy
import warnings

constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

In [ ]:
client, dview, lview = hpc05.start_remote_and_connect(120, folder='~/two_dim_majoranas', timeout=1800)

In [ ]:
hpc05.kill_remote_ipcluster()

In [ ]:
try:
    runner.cancel()
except:
    pass

runner = adaptive.Runner(bl_band, executor=client)
# bl.start_periodic_saver(runner,folder_path,interval=1800)

In [ ]:
runner.live_info()

In [ ]:
def func(k, B, phase):
    import kwant
    import numpy as np
    import scipy.constants
    import cmath
    import sns_system

    constants = dict(
        m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
        hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
        current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
        mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
        exp=cmath.exp,
        cos=cmath.cos,
        sin=cmath.sin
       )

    syst_pars = {'Ll' : 1000,
               'Lr' : 1000,
               'Lm' : 250,
               'Ly' : 4000,
               'a' :  25}

    params = dict(**constants,
                  g_factor = 10,
                  mu = 1.8,
                  alpha = 28,
                  Delta = 0.18,
                  B = B,
                  phase = phase)
    
    syst = sns_system.make_sns_system(**syst_pars)
    bands = kwant.physics.Bands(syst.leads[0], params=params)

#     momenta = np.linspace(-0.5, 0.5, nsamples)
#     energies = [bands(k) for k in momenta]
#     nbands = len(energies[0])
#     band_struct = [(momenta, [energies[k][band] for k in range(nsamples)]) for band in range(nbands)]
    
    return bands(k)

In [ ]:
lrnr = bl.learners[100]
learners = []
for Bphase in lrnr.data.keys():
    (B, phase) = Bphase
    print
    f = partial(func, B=B, phase=phase)
    learner = adaptive_tools.Learner1D(f, bounds=[-np.pi, np.pi])
    learners.append(learner)
    
bl_band = adaptive_tools.BalancingLearner(learners)

In [ ]:
res.shape

In [ ]:

def band_struct(B, phase):
    band_struct = func((B, phase))
    nbands = len(band_struct)
    res = {band : hv.Curve(band_struct[band]) for band in range(nbands)}
    return hv.NdOverlay(res)

In [ ]:
%%timeit
bands = kwant.physics.Bands(syst.leads[0], params=params)

In [ ]:
for learner in bl_band.learners:
    print(f'\tnpoints: {learner.npoints}\tloss: {learner.loss():.2f}')

In [ ]:
learner = bl_band.learners[0]

In [ ]:
%autoreload

In [ ]:
plotting_results.plot_band_result(learner.data.keys(), learner.data.values())

In [ ]:
nsamples = 1001
momenta = np.linspace(-np.pi/2, np.pi/2, nsamples)
energies = [bands(k) for k in momenta]
nbands = len(energies[0])
band_struct = [(momenta, [energies[k][band] for k in range(nsamples)]) for band in range(nbands)]

In [ ]:
learner = adaptive.Learner1D(bands, bounds=[-np.pi, np.pi])

In [ ]:
try:
    runner.cancel()
except:
    pass

runner = adaptive.Runner(learner)
runner.live_info()

In [ ]:
folder_path = '/home/tinkerer/two_dim_majoranas/phase_zeeman_plots/tsoi_True_mu_1.8[51]_a_[5,_15.625]/B_6_Delta_0.18_Ll_1000_Lm_250_Lr_1000_Ly_4000_a_15.625_alpha_28_g_factor_10_mu_1.8_phase_1'
learners = []

mu = np.linspace(0,1.8,51)
a = list([5, 15.625])
transverse_soi = True
syst_pars = {'Ll' : 1000,
           'Lr' : 1000,
           'Lm' : 250,
           'Ly' : 4000,
           'a' :  15.625}

params = dict(g_factor = 10,
              mu = 1.8,
              alpha = 28,
              Delta = 0.18,
              B = 6,
              phase = 1)

def loss(ip):
    from adaptive.learner.learner2D import default_loss, areas
    adaptive.learner.learner2D.np
    loss = default_loss(ip)
    dim = areas(ip).shape[0]
    return 1e8 * loss / dim if dim < 2000 else loss    

for _mu,_a in product(mu, a):
    params['mu'] = _mu
    syst_pars['a'] = _a
    f = partial(distributed_sns.f_adaptive, keys=('B', 'phase'), params=copy(params), 
            syst_pars=copy(syst_pars), transverse_soi=transverse_soi)
    learner = adaptive_tools.Learner2D(f, bounds=[(0, 1.2), (-1.0*np.pi, 1.0*np.pi)], loss_per_triangle=loss)
    
    learner.pars = (_mu,_a)
    learners.append(learner)


bl = adaptive_tools.BalancingLearner(learners)

try:
    bl.load(folder_path)
except:
    pass

In [ ]:
img = bl.learners[100].plot(tri_alpha=0.3)
img

In [ ]:
data = bl.learners[100].data
ydata = data.copy()
xdata = data.keys()

In [ ]:
tap = hv.streams.Tap(source=img, x=0, y=0)
img

In [ ]:
vals = tap.get_param_values()
x = vals[1][1]
y =vals[2][1]
try:
    runner.cancel()
except:
    pass

f = partial(func, B=x, phase=y)
learner = adaptive.Learner1D(f, bounds=[-np.pi, np.pi])
runner = adaptive.Runner(learner)
runner.live_info()

In [ ]:
vals

In [ ]:
# [('name', 'Tap73213'), ('x', 0.6121585257778083), ('y', -3.0997048037181467)]
plotting_results.plot_band_result(learner.data.keys(), learner.data.values())

In [ ]:
# [('name', 'Tap73213'), ('x', 0.6015858385531387), ('y', -0.05642036122871527)]
plotting_results.plot_band_result(learner.data.keys(), learner.data.values())

In [ ]:
tap = hv.streams.Tap(source=img, x=0, y=0)


In [ ]:
def test_f(xy, **params):
    (B, phase) = xy
    
    try:
        runner.cancel()
    except:
        pass
    
    f = partial(func, B=B, phase=phase)
    learner = adaptive.Learner1D(f, bounds=[-np.pi, np.pi])
    runner = adaptive.Runner(learner, executor=client)
    
#     res = hv.Curve((d, x*(d - y)**2), **params, label=f'xy =({x:.2f}, {y:.2f})')
    res = learner.plot()
    res.learner = learner
    res.runner = runner
    return res


dmap = plotting_results.get_linked_plots(test_f, img, data, default_xy=(0,0), extents=(-4,-4,4,4))
img

In [ ]:
dmap

In [ ]:
(x, y) = (0.1, 4)
hv.Points((x, y), extents=(-4.0,0.0,4.0,1.3))

In [ ]:
time.event(t=0.2)

In [ ]:
max(0,1)

In [ ]:

pointer = hv.streams.PointerXY()

def func(a, b, momenta):
    return [a*(momenta-b)**2 for k in momenta]

In [ ]:
hv.DynamicMap(lambda x, y: hv.Curve((momenta, func(a, b, momenta))), streams=[pos])

In [ ]:
curve_opts = dict(line_dash='dashed')
res = {band : hv.Curve(band_struct[band]) for band in range(nbands)}
a = hv.NdOverlay(res)

In [ ]:
cmap =  hv.Cycle('Category20').values
cycle = hv.Cycle([cmap[c%len(cmap)] for c in range(nbands//2)] + 
                 [cmap[(c%len(cmap))] for c in range(nbands//2-1, -1, -1)])

In [ ]:
a.opts(style = {'Curve': {'color': cycle}})

In [ ]:
[(c%len(cmap)) for c in range(nbands//2-1, -1, -1)]

In [ ]:
nbands//2-1

In [ ]:
res = hv.Curve((momenta,momenta))

In [ ]:
a = res.opts(style={'line_dash':'dashed'})

In [ ]:
hv.Overlay([a, a])

In [ ]:
hv.Curve(band_struct[0]).opts(style=curve_opts)

In [ ]:
res = np.array(energies)

In [ ]:
syst_pars_t = {'Ll' : 5,
           'Lr' : 5,
           'Lm' : 250,
           'Ly' : 50,
           'a' :  5}
syst_pars_j = {'Ll' : 5,
           'Lr' : 5,
           'Lm' : 250,
           'Ly' : 500,
           'a' :  5}


params = dict(**constants,
              g_factor = 10,
              mu = 0.18,
              alpha = 28,
              Delta = 0.0,
              B = 0.2,
              phase =np.pi)

syst_j = sns_system.make_junction(**syst_pars_j)
syst_t = sns_system.make_sns_system(**syst_pars_t)
_ = kwant.plot(syst_t)
_ = kwant.plot(syst_j)

In [ ]:
smat_t = kwant.smatrix(syst_t, in_leads=[0], out_leads=[1], energy=0.0, params=params)
N_t = smat_t.num_propagating(1)
T_t = smat_t.transmission(1,0)/N_t

In [ ]:
smat_j = kwant.smatrix(syst_j, energy=0, params=params)
N_j = smat_j.num_propagating(0)
T_j = smat_j.transmission(0,1)/N_j

In [ ]:
def delta(W):
    return (constants['hbar']**2*np.pi**2/(2*constants['m_eff']*(2*W)**2))

T_j*N_j*delta(250)